In [ ]:
import pymatgen

In [ ]:
import ase.db

In [ ]:
import numpy as np

In [ ]:
import collections as coll

In [ ]:
!wget https://cmr.fysik.dtu.dk/_downloads/mp_gllbsc.db

In [ ]:
con = ase.db.connect('mp_gllbsc.db')

In [ ]:
from mpcontribs.io.archieml.mpfile import MPFile  

In [ ]:
s = """
title: GLLB-SC Bandgaps
references.AEM: Advanced Energy Materials, Juli 22, 2014
references.PRA: O. Gritsenko, R. van  Leeuwen, E. van Lenthe, E. J. Baerends, Phys. Rev. A 1995, 51, 1944.  
references.PRB: M. Kuisma, J. Ojanen, J. Enkovaara, T. T. Rantala, Phys. Rev. B 2010, 82, 115106.  

contributor : Technical University of Denmark
authors: Ivano E. Castelli, Falco Hueser, Mohnish Pandey, Hong Li, Kristian S. Thygesen, Brian Seger, Anubhav Jain, Kristin A. Persson, Gerbrand Ceder, and Karsten W. Jacobsen   
explanation: Bandgaps calculated using GLLB-SC potential by Gritsenko, van Leeuwen, van Lenthe, and Baerends (GLLB), adapted by Kuisma et al to account for solids (-SC). The Kohn-Sham gap most basically refers to the energy difference between the bottom of the conduction band and the top of the valence band. The Quasi-Particle gap takes into account the derivative discontinuity of the chemical potential and gives the bandgap as the difference between ionization potenital and electron affinity. 
"""

In [ ]:
mpfile = MPFile.from_string(s)
count = 0
for row in con.select('mpid'):
    d = coll.OrderedDict([])
    d['Kohn-Sham_Bandgap'] = coll.OrderedDict([])
    d['Derivative_Discontinuity'] = coll.OrderedDict([]) 
    d['Quasi-Particle_Bandgap'] = coll.OrderedDict([])
    count = count + 1
    mpid = 'mp-' + str(row.mpid)
    d['Kohn-Sham_Bandgap']['Indirect'] = row.gllbsc_ind_gap - row.gllbsc_disc
    d['Kohn-Sham_Bandgap']['Direct'] = row.gllbsc_dir_gap - row.gllbsc_disc
    d['Derivative_Discontinuity'] = row.gllbsc_disc
    d['Quasi-Particle_Bandgap']['Indirect'] = row.gllbsc_ind_gap
    d['Quasi-Particle_Bandgap']['Direct'] = row.gllbsc_dir_gap
    mpfile.add_hierarchical_data(d, identifier=mpid) 
    if count == 10:
        break

In [ ]:
import pandas as pd
data1 = np.loadtxt('dtu_gllbsc_gaps.txt')
data_txt = pd.DataFrame(data1)
data_txt = data_txt.sort_values(0)
data = []
for row in con.select('mpid'):
    data.append([row.mpid, row.gllbsc_ind_gap - row.gllbsc_disc, row.gllbsc_dir_gap - row.gllbsc_disc, row.gllbsc_disc,
                row.gllbsc_ind_gap, row.gllbsc_dir_gap])

data_db = pd.DataFrame(data)
data_db = data_db.sort_values(0)

data_txt['6'] = 'txt'
data_db['6'] = 'db'
data_all = pd.concat([data_txt,data_db])
data_all = data_all.sort_values(0)
data_all = data_all.reset_index(drop=True)
data_all = data_all.drop(data_all.index[[0]]) 
missing_mp = data_all.drop_duplicates(keep = False, subset = 0)
print(missing_mp)   

In [ ]:
mpfile